Import Needed Modules

In [21]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install git+https://github.com/tensorflow/docs
import collections

import os
import cv2
import numpy as np

import tensorflow as tf

import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed # After installation, this line should work correctly.

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-7hi26v8x
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-7hi26v8x
  Resolved https://github.com/tensorflow/docs to commit 98fa05949ba64f95c4df6565ea915a002a240289
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-2024.7.15.51478-py3-none-any.whl size=182586 sha256=89b6ee8777ea11b3ff1382916cb94e7391ce61cb3e03fbd297147bff06694444
  Stored in directory: /tmp/pip-ephem-wheel-cache-rzgj5gvd/wheels/86/0f/1e/3b62293c8ffd0fd5a49508e6871cdb7554abe9c62afd35ec53
Successfully built tensorflow-docs


Get Data

In [3]:
import zipfile

def list_files_from_zip(zip_path):
    """List the files in each class of the dataset given a local ZIP file.

    Args:
      zip_path: The path to the local ZIP file.

    Returns:
      List of files in each of the classes.
    """
    files = []
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for zip_info in zip_ref.infolist():
            files.append(zip_info.filename)
    return files

URL = "/content/drive/MyDrive/Shop DataSet.zip"
files = list_files_from_zip(URL)

files = [f for f in files if f.endswith('.mp4')]

files[:10]


['Shop DataSet/non shop lifters/shop_lifter_n_0.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_0_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_1_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_10.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_10_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_100.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_100_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_101.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_101_1.mp4']

In [4]:
import os

def get_class(fname):
    split_path = os.path.normpath(fname).split(os.sep)
    class_name = split_path[-2]

    return class_name

In [5]:
NUM_CLASSES = 2
def get_files_per_class(files):

  files_for_class = collections.defaultdict(list)
  for fname in files:
    class_name = get_class(fname)
    files_for_class[class_name].append(fname)
  return files_for_class


files_for_class = get_files_per_class(files)
classes = list(files_for_class.keys())

In [6]:
print('Num classes:', len(classes))
print('Num videos for class[0]:', len(files_for_class[classes[0]]))
print('Num videos for class[1]:', len(files_for_class[classes[1]]))

Num classes: 2
Num videos for class[0]: 531
Num videos for class[1]: 324


In [7]:
print(classes)

['non shop lifters', 'shop lifters']


In [ ]:
files_for_class

In [10]:
import random

def select_subset_of_classes(files_for_class, classes, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):

    files_split = dict()

    for class_name in classes:
        class_files = files_for_class[class_name]
        num_files = len(class_files)

        # Shuffle files to ensure randomness
        random.shuffle(class_files)

        # Determine the number of files for each split
        num_train = int(train_ratio * num_files)
        num_val = int(val_ratio * num_files)
        num_test = num_files - num_train - num_val  # The remaining files go to test

        # Split the files into train, val, and test sets
        train_files = class_files[:num_train]
        val_files = class_files[num_train:num_train + num_val]
        test_files = class_files[num_train + num_val:]

        # Store the subsets for the current class
        files_split[class_name] = {
            'train': train_files,
            'val': val_files,
            'test': test_files
        }

    return files_split


In [15]:
files_split = select_subset_of_classes(files_for_class, classes[:NUM_CLASSES])

print(len(files_split['non shop lifters']['train']))
print(len(files_split['non shop lifters']['val']))
print(len(files_split['non shop lifters']['test']))
print(len(files_split['shop lifters']['train']))
print(len(files_split['shop lifters']['val']))
print(len(files_split['shop lifters']['test']))

424
53
54
259
32
33


In [ ]:
import pandas as pd

# Create empty DataFrames for each split
train_df = pd.DataFrame()
val_df = pd.DataFrame()
test_df = pd.DataFrame()

# Iterate over classes and splits
for class_name, class_data in files_split.items():
    for split_name, files in class_data.items():
        # Create a DataFrame for the current split
        split_df = pd.DataFrame({'filename': files, 'class': class_name})

        # Append to the corresponding DataFrame
        if split_name == 'train':
            train_df = pd.concat([train_df, split_df], ignore_index=True)
        elif split_name == 'val':
            val_df = pd.concat([val_df, split_df], ignore_index=True)
        elif split_name == 'test':
            test_df = pd.concat([test_df, split_df], ignore_index=True)

# Print the DataFrames
print(train_df)
print(val_df)
print(test_df)

In [17]:

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 683
Total videos for testing: 87


,filename,class
589,Shop DataSet/shop lifters/videmmmmmmsss_86.mp4,shop lifters
46,Shop DataSet/non shop lifters/shop_lifter_n_18...,non shop lifters
387,Shop DataSet/non shop lifters/shop_lifter_n_18...,non shop lifters
74,Shop DataSet/non shop lifters/shop_lifter_n_12...,non shop lifters
573,Shop DataSet/shop lifters/shop_lifter_97.mp4,shop lifters
228,Shop DataSet/non shop lifters/shop_lifter_n_11...,non shop lifters
388,Shop DataSet/non shop lifters/videppppsss_18.mp4,non shop lifters
376,Shop DataSet/non shop lifters/shop_lifter_n_92...,non shop lifters
131,Shop DataSet/non shop lifters/shop_lifter_n_15...,non shop lifters
15,Shop DataSet/non shop lifters/shop_lifter_n_51...,non shop lifters


Set Hyperparameters

In [18]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 5
NUM_FEATURES = 2048

Create Helper Functions

In [19]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [22]:
def build_feature_extractor():
    feature_extractor = keras.applications.resnet50.ResNet50(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [23]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["class"]))

print(label_processor.get_vocabulary())

['non shop lifters', 'shop lifters']


In [24]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2 to filter out INFO and WARNING, 3 to filter out INFO, WARNING, and ERROR

In [25]:
import sys
import os
import contextlib
import tqdm

@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [27]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["filename"].values.tolist()
    labels = df["class"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in tqdm.tqdm(enumerate(video_paths), total=len(video_paths)):
        # Gather all its frames and add a batch dimension.
#         frames = load_video(os.path.join(root_dir, path))
        frames = load_video( path)

        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                with suppress_stdout():

                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

100%|██████████| 87/87 [00:00<00:00, 11335.25it/s]

Frame features in train set: (683, 5, 2048)
Frame masks in train set: (683, 5)


In [28]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    #filepath = "/tmp/video_classifier"
    #checkpoint = keras.callbacks.ModelCheckpoint(
    #    filepath, save_weights_only=True, save_best_only=True, verbose=1
    #)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        #callbacks=[checkpoint],
    )

    #seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.8859 - loss: 0.6903 - val_accuracy: 0.0000e+00 - val_loss: 0.7081
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8858 - loss: 0.6790 - val_accuracy: 0.0000e+00 - val_loss: 0.7233
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8887 - loss: 0.6678 - val_accuracy: 0.0000e+00 - val_loss: 0.7385
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9059 - loss: 0.6555 - val_accuracy: 0.0000e+00 - val_loss: 0.7537
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9011 - loss: 0.6451 - val_accuracy: 0.0000e+00 - val_loss: 0.7691
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8929 - loss: 0.6358 - val_accuracy: 0.0000e+00 - val_loss: 0.7844
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8882 - loss: 0.6268 - val_accuracy: 0.0000e+00 - val_loss: 0.7996
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8826 - los

In [29]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, duration=100)
    return embed.embed_file("animation.gif")

Test Video

In [30]:
test_video = np.random.choice(test_df["filename"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])

Test video path: Shop DataSet/shop lifters/videppppsss_6.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
  non shop lifters: 84.84%
  shop lifters: 15.16%


ValueError: Image data must be a sequence of ndimages.

In [31]:
test_frames

array([], dtype=float64)